A fundamental concept in Genki Signals is the (circular) buffer:

In [ ]:
from genki_signals.buffers import DataFrameBuffer

buffer = DataFrameBuffer(size=400)

A `DataFrameBuffer` is similar to a `pandas` `DataFrame` apart from the finite size. If we add more than 400 elements to it, the first ones will be automatically deleted. Genki Signals also provides `ArrayBuffer` (similar to a `numpy` array) and `TensorBuffer` (like a `pytorch` tensor). 

A buffer without data is like a book without words, so let's generate some data! We'll make a simple sine wave generator to begin with:

In [1]:
from genki_signals.data_sources import SineWave

source = SineWave(
    freq=[1, 3],
    amplitude=[5, 1],
    phase=[0, 0],
    sampling_freq=100,
)

source.signal_names

# ['sine_0', 'sine_1']

ModuleNotFoundError: No module named 'genki_signals'

A few comments:

* `freq`, `amplitude`, and `phase` all accept a list of two elements. This means the generator is actually generating _two_ independent waves with different frequencies and amplitudes (the phase angles are the same in this case)
* `freq` and `sampling_freq` are measured in cycles and samples _per second_ (Hz) - note that time here is not some abstract x-axis but actually real time. Once this data source is put in the right context and started it will generate 100 samples per second of two sine waves with 1 and 3 cycles per second. This data is generated in a separate thread, regardless of wether it is read or used for anything.
* `freq`, `amplitude`, and `phase` are all specific to sine wave generation, whereas `sampling_freq` is more general - this argument is present in more data sources
* The two sine waves are given __names__ - when we define some signal processing on the waves we will need to refer to them by name 

While the sine wave data source is useful as an example, data sources that are entirely deterministic are in general not very useful. Usually data sources represent some kind of _input_, e.g. mouse movements or Wave sensor readings:

In [2]:
from genki_signals.data_sources import MouseDataSource, WaveDataSource

mouse_source = MouseDataSource(sampling_freq=100)
wave_source = WaveDataSource(ble_address)

ModuleNotFoundError: No module named 'genki_signals'

The `MouseDataSource` reads the position of the mouse, and the `WaveDataSource` streams sensor data from a Wave ring. Note that `WaveDataSource` takes no `sampling_freq` argument, this is because Wave operates on it's own clock, and the sampling rate is determined by firmware. 

Anyways, let's go back to the sine wave example. We currently have a data source and a dataframe buffer, but we haven't connected them. One way to do so is to make the data source populate the buffer in another thread to avoid blocking the main jupyter thread:

In [ ]:
import time
from threading import Thread

def run_source():
    source.start()
    while True:
        buffer.append(source.read())
        time.sleep(1 / 25)
        
t = Thread(target=run_source)
t.run()

As soon as the `start` method of the data source is called, it starts generating data. Calling `source.read()` returns all data that the source has generated since the last call to `read`. In this case we are calling `read` 25 times a second, but the data source is generating data at 100hz, so we should expect `read` to return 4 data points on average. We have in effect introduced yet another frequency that is relevant which we can call _update frequency_. This is a useful distinction. If we are streaming very high frequency data (e.g. audio) through a websocket, we can lower the update frequency and chunk the data to fewer TCP transmissions. This data is generally meant to be plotted in real time so the other limit of the update frequency is human perception, if graphs appear chunky, the update frequency can be increased.

Speaking of graphs, we can now visualise the data as it is generated:

In [4]:
buffer.plot() # This should return a widget with a real-time line chart

NameError: name 'buffer' is not defined

This is neat, but the real power of Genki Signals comes from using derived signals:

In [3]:
import genki_signals.signals as s

derived_signals = [
    s.Sum('sine_0', 'sine_1', name='composite_sine')
    s.DFT('composite_sine', name='composite_sine_spectrum', window_size=128, window_overlap=32)
]

ModuleNotFoundError: No module named 'genki_signals'

What are `derived_signals`? It specifies a configuration of derived signals in the context of some data source. Adding `sine_0` and `sine_1` is meaningless without knowing what `sine_0` and `sine_1` are. 

What do we want from an object like `derived_signals`?

* It represents a DAG of time-series operations, each operation can take source signals or results of other operations as input. Names are important!
* These should work both offline and online (real time). They are _causal_ in other words.
* They should be deterministic - reproducible from the specification only and can only depend on local state.
* Determinism means they are serializable to e.g. JSON - this will be passed from frontend to backend in a web app 
* We can view it as a torch module (and sklearn pipeline?) - which is then serializable as ONNX
* This is tensor backend-agnostic. Each signal should be capable of working with numpy arrays, torch tensors, etc.
* Each signal in the list is a function that can operate independently given some data.

We are still not computing any of the derived signals. If we want to start all of this - a thing which computes two sine waves, adds them together and computes the DFT, we could:

* Start the source in a separate thread
* Manage all the names and compute all the signals at each step
* Keep all the results in a buffer

This is quite tedious, but thankfully this is exactly what a `System` does:

In [5]:
from genki_signals.system import System

system = System(
    source=source,
    derived_signals=derived_signals,
    buffer_len=400,
    tensor_backend='numpy'
)

ModuleNotFoundError: No module named 'genki_signals'

Most of this is straightforward, but notice the `tensor_backend`. Everything we have defined up to this point is agnostic as to which library is used for vectorised operations. The value of `tensor_backend` can be `'numpy'` for numpy arrays or `'pytorch'` for PyTorch tensors. 

In [6]:
source.stop() # the source is currently running, we need to stop it to be able to start it again
system.start()

system.buffer.plot(plot_type='spectrogram', signal='compmosite_sine_spectrum')

NameError: name 'source' is not defined

Setting `plot_type` changes the type of plot created. Genki signals includes a few widgets for data visualisation. Here are a few examles:

In [7]:
mouse_system = System(source=mouse_source)
mouse_system.start()
mouse_system.buffer.plot(ploy_type='trace_2d')

NameError: name 'System' is not defined

In [8]:
wave_system = System(
    source=wave_source,
    derived_signals=[
        s.SamplingRate(),
        s.EulerAngle('current_pose', name='euler_pose') # Or whatever is needed for 3D cube to work
    ]
)

wave_system.start()
wave_system.buffer.plot(plot_type='format_text', signal='sampling_rate', text='Wave sampling rate: {}')

NameError: name 'System' is not defined

In [9]:
wave_system.buffer.plot(plot_type='cube_3d', signal='euler_pose')

NameError: name 'wave_system' is not defined

Other things to add to demo:

* Show model inference with histogram
* Demonstrate signal spec serialisation to e.g. ONNX
* More signal processing magic - e.g. modulate a signal with noise and delay and demodulate
